In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from pandas import MultiIndex, Int64Index

from sklearn.model_selection import KFold

import optuna
import pandas as pd
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection

from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag
from sklearn.model_selection import train_test_split
import bokeh



/usr/local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/var/folders/32/zv3nm8_12xb9ytd7vk7nnr6w0000gn/T/ipykernel_47053/74542634.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#!pip3 install xgboost
#!pip3 install optuna
#!pip install autogluon --upgrade
# !pip install optuna
# !pip install scipy
# !pip install autogluon
# !pip install xgboost
# !python3.9 -m pip install --upgrade pip
#!pip install bokeh==2.0.1
#!pip install jupyter --upgrade


In [3]:
df_train = pd.read_csv('../data/train_encoded_1.csv')
df_test = pd.read_csv('../data/test_encoded_1.csv')

In [4]:
np.random.seed(0)
train, valid = train_test_split(df_train, test_size=0.2)

In [5]:
combined = df_train.drop(['price', 'size_sqft'], axis=1)
X_train = train.drop(['price', 'size_sqft'], axis=1)

X_valid = valid.drop(['price', 'size_sqft'], axis=1)
y_valid = valid['price_per_sqft']

df_test_size_sqft = df_test['size_sqft'].copy()
df_test= df_test.drop(['size_sqft'], axis=1)

metric_reg = 'mean_absolute_error'
label_reg = 'price_per_sqft'
train_data_reg = TabularDataset(X_train) 
valid_data_reg = TabularDataset(X_valid)
test_data_reg = TabularDataset(df_test)
combined = TabularDataset(combined)



Tabular Predictor

In [6]:
predictor = TabularPredictor(label = label_reg, eval_metric = metric_reg)
hyperparameters = {
 'NN_TORCH': {},
 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}],
 'XT':{}
}

time_limit = 30*60  # train various models for ~5 min
num_trials = 3  # try at most 3 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

excluded_model_types = ['FASTAI', 'KNN', 'RF', 'XGB']
predictor_reg = predictor.fit(combined, excluded_model_types = excluded_model_types
,hyperparameters=hyperparameters
,hyperparameter_tune_kwargs=hyperparameter_tune_kwargs, presets='best_quality'
)


No path specified. Models will be saved in: "AutogluonModels/ag-20221028_023652/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221028_023652/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    18424
Train Data Columns: 31
Label Column: price_per_sqft
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (4525.797460167968, 461.7295308187673, 1811.07817, 965.70493)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
/usr/local/lib/python3.9/site-packages/autogluon/tabular/learner/de

  0%|          | 0/3 [00:00<?, ?it/s]

[1000]	valid_set's l1: 161.308
[2000]	valid_set's l1: 157.268
[3000]	valid_set's l1: 155.77
[4000]	valid_set's l1: 154.747
[5000]	valid_set's l1: 154.259
[6000]	valid_set's l1: 153.845
[7000]	valid_set's l1: 153.713
[8000]	valid_set's l1: 153.731
[9000]	valid_set's l1: 153.829
[1000]	valid_set's l1: 152.737
[2000]	valid_set's l1: 151.687
[3000]	valid_set's l1: 151.331
[4000]	valid_set's l1: 151.159
[5000]	valid_set's l1: 151.209
[1000]	valid_set's l1: 162.246
[2000]	valid_set's l1: 157.808
[3000]	valid_set's l1: 156.195
[4000]	valid_set's l1: 155.429
[5000]	valid_set's l1: 155.154
[6000]	valid_set's l1: 155.117
[7000]	valid_set's l1: 155.141
[8000]	valid_set's l1: 155.121


Fitted model: LightGBMXT_BAG_L1/T1 ...
	-153.6036	 = Validation score   (-mean_absolute_error)
	11.64s	 = Training   runtime
	0.48s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1/T2 ...
	-151.1279	 = Validation score   (-mean_absolute_error)
	12.0s	 = Training   runtime
	0.36s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1/T3 ...
	-154.9918	 = Validation score   (-mean_absolute_error)
	16.03s	 = Training   runtime
	0.55s	 = Validation runtime
Hyperparameter tuning model: ExtraTrees_BAG_L1 ...
	No hyperparameter search space specified for ExtraTrees. Skipping HPO. Will train one model based on the provided hyperparameters.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/autogluon/core/trainer/abstract_trainer.py", line 1380, in _train_single_full
    hpo_models, hpo_results = model.hyperparameter_tune(
  File "/usr/local/lib/python3.9/site-packages/autogluon/core/models/abstract/abstract_model.py", line 1001, in hyperparameter_tune
    ret

[1000]	valid_set's l1: 147.955
[2000]	valid_set's l1: 144.087
[3000]	valid_set's l1: 142.685
[4000]	valid_set's l1: 142.201
[5000]	valid_set's l1: 142.093
[6000]	valid_set's l1: 142.112
[7000]	valid_set's l1: 142.179
[1000]	valid_set's l1: 148.792
[2000]	valid_set's l1: 146.745
[3000]	valid_set's l1: 146.16
[4000]	valid_set's l1: 145.635
[5000]	valid_set's l1: 145.596
[6000]	valid_set's l1: 145.629
[1000]	valid_set's l1: 156.044
[2000]	valid_set's l1: 153.007
[3000]	valid_set's l1: 152.175
[4000]	valid_set's l1: 152.046
[5000]	valid_set's l1: 151.865
[6000]	valid_set's l1: 151.739
[7000]	valid_set's l1: 151.712
[8000]	valid_set's l1: 151.706
[9000]	valid_set's l1: 151.71
[1000]	valid_set's l1: 143.971
[2000]	valid_set's l1: 140.902
[3000]	valid_set's l1: 139.364
[4000]	valid_set's l1: 138.986
[5000]	valid_set's l1: 139.179
[1000]	valid_set's l1: 154.494
[2000]	valid_set's l1: 151.822
[3000]	valid_set's l1: 150.864
[4000]	valid_set's l1: 150.622
[5000]	valid_set's l1: 150.591
[1000]	val

...

	-149.1997	 = Validation score   (-mean_absolute_error)
	165.81s	 = Training   runtime
	7.38s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L1/T1 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-148.7736	 = Validation score   (-mean_absolute_error)
	524.96s	 = Training   runtime
	0.38s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L1/T2 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-146.2259	 = Validation score   (-mean_absolute_error)
	511.7s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L1/T3 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-154.5445	 = Validation score   (-mean_absolute_error)
	1464.62s	 = Training   runtime
	2.3s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-137.6912	 = Validation score   (-mean_absolute_error)
	0.34s	 = Training   runtime
	0.0s	 = Validati

  0%|          | 0/3 [00:00<?, ?it/s]

Fitted model: LightGBMXT_BAG_L2/T1 ...
	-137.3028	 = Validation score   (-mean_absolute_error)
	0.85s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L2/T2 ...
	-138.2441	 = Validation score   (-mean_absolute_error)
	1.75s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L2/T3 ...
	-139.2613	 = Validation score   (-mean_absolute_error)
	1.95s	 = Training   runtime
	0.05s	 = Validation runtime
Hyperparameter tuning model: ExtraTrees_BAG_L2 ...
	No hyperparameter search space specified for ExtraTrees. Skipping HPO. Will train one model based on the provided hyperparameters.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/autogluon/core/trainer/abstract_trainer.py", line 1380, in _train_single_full
    hpo_models, hpo_results = model.hyperparameter_tune(
  File "/usr/local/lib/python3.9/site-packages/autogluon/core/models/abstract/abstract_model.py", line 1001, in hyperparameter_tune
    retur

[1000]	valid_set's l1: 147.409


	-145.4989	 = Validation score   (-mean_absolute_error)
	16.78s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L2/T3 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-145.8247	 = Validation score   (-mean_absolute_error)
	12.19s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L2/T1 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-143.0367	 = Validation score   (-mean_absolute_error)
	212.09s	 = Training   runtime
	0.36s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L2/T2 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-140.9164	 = Validation score   (-mean_absolute_error)
	263.3s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L2/T3 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-146.5638	 = V

In [11]:
#/Users/nishita/Documents/Semester 1/CS5228/project/notebooks/AutogluonModels/ag-20221027_054028
#This is with train and valid split data, with best_quality preset.
#folderName = "AutogluonModels/ag-20221028_001415/"
#This is with all of the training data available and with psf_bbox in float instead of int, results should be better slightly
folderName = "AutogluonModels/ag-20221028_023652/"
predictor = TabularPredictor.load(folderName)
predictor.fit_hyperparameters_

{'NN_TORCH': {},
 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}],
 'XT': {}}

In [12]:
predictor.evaluate(valid_data_reg)

Evaluation: mean_absolute_error on test data: -104.54879893310817
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -104.54879893310817,
    "root_mean_squared_error": -212.52178955124276,
    "mean_squared_error": -45165.51103406272,
    "r2": 0.9508637819070908,
    "pearsonr": 0.9751752886937128,
    "median_absolute_error": -53.01186699478194
}


{'mean_absolute_error': -104.54879893310817,
 'root_mean_squared_error': -212.52178955124276,
 'mean_squared_error': -45165.51103406272,
 'r2': 0.9508637819070908,
 'pearsonr': 0.9751752886937128,
 'median_absolute_error': -53.01186699478194}

In [14]:
#With target encoding and less one hot encoding.
predictor = TabularPredictor.load(folderName)
results = predictor.fit_summary(show_plot=True)

def writePredictionsToFile(modelName, filename, folder = folderName, refit = False):
    if(refit == True):
        modelName = modelName +  '_FULL'    
    y_pred = predictor.predict(test_data_reg, model=modelName)
    yTest = y_pred * df_test_size_sqft
    yTest = pd.DataFrame(yTest, columns=['Predicted'], index=df_test.index)
    yTest.to_csv(f'{folder}/predictions/{filename}.csv')


*** Summary of fit() ***
Estimated performance of each model:
                       model   score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        WeightedEnsemble_L2 -137.691185       7.504357  2641.846519                0.000802           0.338653            2       True          7
1        WeightedEnsemble_L3 -138.828713      16.098094  3754.387172                0.000736           0.331317            3       True         14
2   NeuralNetTorch_BAG_L2/T2 -140.916443      15.091498  3070.618042                0.207525         263.295259            2       True         12
3   NeuralNetTorch_BAG_L2/T1 -143.036661      15.242933  3019.410966                0.358960         212.088183            2       True         11
4       LightGBMXT_BAG_L2/T1 -144.672149      15.056707  2817.525470                0.172734          10.202686            2       True          8
5       LightGBMXT_BAG_L1/T2 -145.368549       1.340513 

In [15]:
predictor.leaderboard(valid_data_reg)
predictor.evaluate(valid_data_reg)

                       model  score_test   score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       LightGBMXT_BAG_L1/T2  -91.154465 -145.368549        2.270393       1.340513    60.481916                 2.270393                1.340513          60.481916            1       True          2
1       LightGBMXT_BAG_L1/T1 -102.339094 -147.587589        5.158992       3.253439    79.738326                 5.158992                3.253439          79.738326            1       True          1
2   NeuralNetTorch_BAG_L2/T2 -103.128439 -140.916443       20.328937      15.091498  3070.618042                 0.296184                0.207525         263.295259            2       True         12
3        WeightedEnsemble_L2 -104.548799 -137.691185       11.549747       7.504357  2641.846519                 0.005994                0.000802           0.338653            2       True          7


Evaluation: mean_absolute_error on test data: -104.54879893310817
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -104.54879893310817,
    "root_mean_squared_error": -212.52178955124276,
    "mean_squared_error": -45165.51103406272,
    "r2": 0.9508637819070908,
    "pearsonr": 0.9751752886937128,
    "median_absolute_error": -53.01186699478194
}


{'mean_absolute_error': -104.54879893310817,
 'root_mean_squared_error': -212.52178955124276,
 'mean_squared_error': -45165.51103406272,
 'r2': 0.9508637819070908,
 'pearsonr': 0.9751752886937128,
 'median_absolute_error': -53.01186699478194}

In [16]:
writePredictionsToFile(modelName = 'LightGBMXT_BAG_L1/T2', filename = 'LightGBMXT_BAG_L1_T2')


In [17]:
writePredictionsToFile(modelName = 'NeuralNetTorch_BAG_L2/T2', filename = 'NeuralNetTorch_BAG_L2_T2')
#writePredictionsToFile('RandomForestMSE')
#writePredictionsToFile('LightGBMLarge')
writePredictionsToFile(modelName = 'WeightedEnsemble_L2', filename = 'WeightedEnsemble_L2')
##writePredictionsToFile('XGBoost')
#writePredictionsToFile(modelName = 'ExtraTreesMSE', filename = 'ExtraTreesMSE')
#writePredictionsToFile('LGBModel')
#writePredictionsToFile(modelName = 'LightGBMXT_BAG_L2/T2', filename = 'LightGBMXT')
#writePredictionsToFile(modelName = 'NeuralNetTorch/a2acf_00004', filename = 'NeuralNetTorch')
#writePredictionsToFile('NeuralNetFastAI')
#writePredictionsToFile('KNeighborsDist')
##writePredictionsToFile('KNeighborsUnif')

In [ ]:
#predictor.refit_full()

In [ ]:
modelName = 'CatBoost_BAG_L2/T3'
y_pred = predictor.predict(valid_data_reg, model=modelName)
yTest = y_pred * valid['size_sqft']
yTest = pd.DataFrame(yTest, columns=['Predicted'], index=valid.index)
    

In [ ]:
yTest.to_csv(f'../predictions/valid_data.csv')


In [ ]:
yTest.shape

In [ ]:
actual = valid['price']
pred = yTest['Predicted']

valid_error = mean_squared_error(actual, pred, squared=False)
    #print(modelName)
print(f"RMSE: {valid_error}")
print(f'Percentage error for validation data {((abs(pred - actual)/actual)*100).mean()}')
